
# Data Cleaning Walkthrough (East African E‑Commerce Dataset)

This notebook will walk you through the process of **cleaning and preparing a real-world dataset** step by step — beginner friendly.

We will cover:
1. Loading and exploring the dataset  
2. Renaming and standardizing columns  
3. Handling missing values  
4. Removing duplicates  
5. Cleaning text, numbers, and dates  
6. Detecting and handling outliers  
7. Feature engineering  
8. Exporting the cleaned dataset  


In [1]:
!pip install  pandas numpy matplotlib seaborn scikit-learn 

In [2]:
!pip install Pillow --upgrade

In [3]:

# Step 1: Import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Display settings for pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)


In [4]:

# Step 2: Load the dataset
file_path = "eastafrican-ecommerce-dataset.csv"
df = pd.read_csv(file_path, on_bad_lines="skip", engine="python")
print(f"Raw shape: {df.shape}  (rows, cols)")# Preview the first few rows
df.head()


Raw shape: (68, 12)  (rows, cols)


,CustomerID,FirstName,LastName,Email,RegistrationDate,Country,City,TotalSpent,Currency,Age,PhoneNumber,LastLoginDate
0,CUST001,david,Okello,david.okello@email.com,15/01/2022,Uganda,Kampala,150000,UGX,34.0,2.567721e+11,20/08/2025
1,CUST002,Esther,Wanjiku,esther.w@email.com,2022-02-20,Kenya,Nairobi,5500,KES,28.0,7.123457e+08,21/08/2025
2,CUST004,Fatuma,Juma,fatuma.juma@work.net,10/04/2022,Tanzania,Dar es Salaam,80000,TZS,31.0,2.557551e+11,15/07/2025
3,CUST005,DAVID,OKELLO,david.okello@email.com,12/05/2022,UG,Kla,75000,Shs,34.0,7.721235e+08,NaN
4,CUST001,david,Okello,david.okello@email.com,15/01/2022,Uganda,Kampala,150000,UGX,34.0,2.567721e+11,20/08/2025


In [5]:

# Step 3: Basic info and summary statistics
df.info()

df_fuzzy = df.copy()  # Create a copy for fuzzy matching later


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CustomerID        68 non-null     object 
 1   FirstName         68 non-null     object 
 2   LastName          68 non-null     object 
 3   Email             67 non-null     object 
 4   RegistrationDate  68 non-null     object 
 5   Country           68 non-null     object 
 6   City              68 non-null     object 
 7   TotalSpent        68 non-null     int64  
 8   Currency          65 non-null     object 
 9   Age               67 non-null     float64
 10  PhoneNumber       67 non-null     float64
 11  LastLoginDate     35 non-null     object 
dtypes: float64(2), int64(1), object(9)
memory usage: 6.5+ KB


In [6]:

# Quick descriptive statistics for numerical columns
df.describe(include='all').transpose()


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
CustomerID,68,65,CUST001,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FirstName,68,65,david,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LastName,68,58,Okello,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Email,67,63,david.okello@email.com,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RegistrationDate,68,22,15/01/2022,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Country,68,8,Kenya,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,68,16,Nairobi,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TotalSpent,68.0,NaN,NaN,NaN,92575.588235,213289.668546,180.0,6125.0,20000.0,95750.0,1250000.0
Currency,65,7,KES,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,67.0,NaN,NaN,NaN,3743121283.880597,30638767858.47826,21.0,29.0,34.0,40.0,250789123456.0


Convert the RegistrationDate and LastLoginDate columns to a consistent datetime format, such as YYYY-MM-DD.

In [7]:
df[ ['LastLoginDate' ,'RegistrationDate']].head()

,LastLoginDate,RegistrationDate
0,20/08/2025,15/01/2022
1,21/08/2025,2022-02-20
2,15/07/2025,10/04/2022
3,NaN,12/05/2022
4,20/08/2025,15/01/2022


In [8]:
df['RegistrationDate'] = pd.to_datetime(df['RegistrationDate'], dayfirst=True, errors='coerce')
df['LastLoginDate'] = pd.to_datetime(df['LastLoginDate'], dayfirst=True, errors='coerce')
df[ ['LastLoginDate' ,'RegistrationDate']].head()

,LastLoginDate,RegistrationDate
0,2025-08-20,2022-01-15
1,2025-08-21,NaT
2,2025-07-15,2022-04-10
3,NaT,2022-05-12
4,2025-08-20,2022-01-15


Standardize Phone Number Format and Data Type

In [9]:
df['PhoneNumber'] = df['PhoneNumber'].astype(str).str.replace(r'\D', '', regex=True)

# Preview the standardized phone numbers
df[['PhoneNumber']].head()

,PhoneNumber
0,2567721234560
1,7123456780
2,2557551234560
3,7721234560
4,2567721234560


In [10]:
# Add '0' to PhoneNumber only if it starts with '7', ignore others
df['PhoneNumber'] = df['PhoneNumber'].apply(lambda x: '0' + x if x.startswith('7') else x)

# Preview the standardized phone numbers
df[['PhoneNumber']].head()

,PhoneNumber
0,2567721234560
1,07123456780
2,2557551234560
3,07721234560
4,2567721234560


In [11]:
df.head()

,CustomerID,FirstName,LastName,Email,RegistrationDate,Country,City,TotalSpent,Currency,Age,PhoneNumber,LastLoginDate
0,CUST001,david,Okello,david.okello@email.com,2022-01-15,Uganda,Kampala,150000,UGX,34.0,2567721234560,2025-08-20
1,CUST002,Esther,Wanjiku,esther.w@email.com,NaT,Kenya,Nairobi,5500,KES,28.0,07123456780,2025-08-21
2,CUST004,Fatuma,Juma,fatuma.juma@work.net,2022-04-10,Tanzania,Dar es Salaam,80000,TZS,31.0,2557551234560,2025-07-15
3,CUST005,DAVID,OKELLO,david.okello@email.com,2022-05-12,UG,Kla,75000,Shs,34.0,07721234560,NaT
4,CUST001,david,Okello,david.okello@email.com,2022-01-15,Uganda,Kampala,150000,UGX,34.0,2567721234560,2025-08-20


NEXT WE NEED TO ENSURE CONSISTTENCY IN THE COUNTRY AND CITY FIELDS 

In [12]:
print("\nUnique values in 'City' column:")
df['City'].unique()


Unique values in 'City' column:


array(['Kampala', 'Nairobi', 'Dar es Salaam', 'Kla', 'Entebbe', 'Niarobi',
       'Kmapala', 'Nrb', 'Mbarara', 'Kigali', 'Gulu', 'Kisumu', 'Arusha',
       'Eldoret', 'Mombasa', 'Jinja'], dtype=object)

In [13]:
print("Unique values in 'Country' column:")

df['Country'].unique()


Unique values in 'Country' column:


array(['Uganda', 'Kenya', 'Tanzania', 'UG', 'KE', 'uganda', 'Rwanda',
       'UGANDA'], dtype=object)

In [14]:
# Create mapping dictionaries based on our discovery
country_mapping = {'ug': 'Uganda', 'ke': 'Kenya', 'UGANDA': 'Uganda'}
city_mapping = {'kla': 'Kampala', 'nrb': 'Nairobi', 'niarobi': 'Nairobi', 'kmapala': 'Kampala'}

# Apply the mapping to the relevant columns
df['Country'] = df['Country'].replace(country_mapping)
df['City'] = df['City'].replace(city_mapping)

# Standardize the casing for uniformity
df['Country'] = df['Country'].str.title()
df['City'] = df['City'].str.title()
df['FirstName'] = df['FirstName'].str.title()
df['LastName'] = df['LastName'].str.title()


In [15]:
print("Unique values in 'Country' column:")

df['Country'].unique()


Unique values in 'Country' column:


array(['Uganda', 'Kenya', 'Tanzania', 'Ug', 'Ke', 'Rwanda'], dtype=object)

In [16]:
print("\nUnique values in 'City' column:")
df['City'].unique()


Unique values in 'City' column:


array(['Kampala', 'Nairobi', 'Dar Es Salaam', 'Kla', 'Entebbe', 'Niarobi',
       'Kmapala', 'Nrb', 'Mbarara', 'Kigali', 'Gulu', 'Kisumu', 'Arusha',
       'Eldoret', 'Mombasa', 'Jinja'], dtype=object)

In [17]:
!pip install fuzzywuzzy

In [18]:
import pandas as pd
from fuzzywuzzy import process
from io import StringIO



# Standardize casing first to improve matching accuracy
df_fuzzy['City'] = df_fuzzy['City'].str.title().str.strip()

# A list of our known, correct city names for matching
correct_cities = ['Kampala', 'Nairobi', 'Kigali', 'Dar Es Salaam', 'Entebbe']

print("--- Step 1: Unique Cities BEFORE Fuzzy Matching ---")
print(df_fuzzy['City'].unique())

def fuzzy_match_and_correct(city_name, correct_list):
    """
    Finds the best fuzzy match for a city name from a list of correct names.
    Returns the corrected city name if the match score is high (>= 80).
    """
    if pd.isna(city_name):
        return city_name
    
    best_match = process.extractOne(city_name, correct_list)
    
    # Print the matching process to demonstrate the logic
    print(f"Processing '{city_name}': Best match is '{best_match[0]}' with score {best_match[1]}")
    
    if best_match[1] >= 80: # Check if the score is 80 or higher
        return best_match[0] # Return the best match name
    else:
        return city_name # Return original name if no good match is found


--- Step 1: Unique Cities BEFORE Fuzzy Matching ---
['Kampala' 'Nairobi' 'Dar Es Salaam' 'Kla' 'Entebbe' 'Niarobi' 'Kmapala'
 'Nrb' 'Mbarara' 'Kigali' 'Gulu' 'Kisumu' 'Arusha' 'Eldoret' 'Mombasa'
 'Jinja']


/home/jericho/Documents/venv/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [19]:

print("\n--- Step 2: Running Fuzzy Match Process ---")
# Apply the fuzzy matching function to the 'City' column
df_fuzzy['City'] = df_fuzzy['City'].apply(lambda x: fuzzy_match_and_correct(x, correct_cities))



--- Step 2: Running Fuzzy Match Process ---
Processing 'Kampala': Best match is 'Kampala' with score 100
Processing 'Nairobi': Best match is 'Nairobi' with score 100
Processing 'Dar Es Salaam': Best match is 'Dar Es Salaam' with score 100
Processing 'Kla': Best match is 'Kampala' with score 60
Processing 'Kampala': Best match is 'Kampala' with score 100
Processing 'Entebbe': Best match is 'Entebbe' with score 100
Processing 'Niarobi': Best match is 'Nairobi' with score 86
Processing 'Kmapala': Best match is 'Kampala' with score 86
Processing 'Nrb': Best match is 'Nairobi' with score 60
Processing 'Nairobi': Best match is 'Nairobi' with score 100
Processing 'Mbarara': Best match is 'Kampala' with score 43
Processing 'Nairobi': Best match is 'Nairobi' with score 100
Processing 'Kigali': Best match is 'Kigali' with score 100
Processing 'Gulu': Best match is 'Kigali' with score 45
Processing 'Nairobi': Best match is 'Nairobi' with score 100
Processing 'Nairobi': Best match is 'Nairobi' wi

In [20]:

print("\n--- Step 3: Unique Cities AFTER Fuzzy Matching ---")
df_fuzzy['City'].unique()




--- Step 3: Unique Cities AFTER Fuzzy Matching ---


array(['Kampala', 'Nairobi', 'Dar Es Salaam', 'Kla', 'Entebbe', 'Nrb',
       'Mbarara', 'Kigali', 'Gulu', 'Kisumu', 'Arusha', 'Eldoret',
       'Mombasa', 'Jinja'], dtype=object)


### Step 5: Handling Missing Values
- We check how many missing values exist.  
- Then decide: drop, fill, or keep them.


In [21]:

# Check missing values
df.isnull().sum()


CustomerID           0
FirstName            0
LastName             0
Email                1
RegistrationDate     2
Country              0
City                 0
TotalSpent           0
Currency             3
Age                  1
PhoneNumber          0
LastLoginDate       33
dtype: int64

In [22]:
df.dropna(subset=['Email'], inplace=True)


In [23]:

for col in df.select_dtypes(include=[np.number]).columns:
    df[col].fillna(df[col].median(), inplace=True)

for col in df.select_dtypes(include=['object']).columns:
    df[col].fillna("Unknown", inplace=True)


/tmp/ipykernel_28435/1957888083.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/tmp/ipykernel_28435/1957888083.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 


### Step 6: Remove Duplicates


In [24]:

# Drop duplicate rows if any
df.drop_duplicates(inplace=True)
df.shape


(64, 12)


### Step 7: Cleaning Text, Numbers, and Dates
- Strip extra spaces  
- Standardize casing  
- Parse dates correctly  


In [25]:

# Example: strip spaces and lowercase for string columns
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.strip().str.lower()


In [26]:

# Convert potential date columns
for col in df.columns:
    if "date" in col:
        df[col] = pd.to_datetime(df[col], errors='coerce')

df.head()


,CustomerID,FirstName,LastName,Email,RegistrationDate,Country,City,TotalSpent,Currency,Age,PhoneNumber,LastLoginDate
0,cust001,david,okello,david.okello@email.com,2022-01-15,uganda,kampala,150000,ugx,34.0,2567721234560,2025-08-20
1,cust002,esther,wanjiku,esther.w@email.com,NaT,kenya,nairobi,5500,kes,28.0,07123456780,2025-08-21
2,cust004,fatuma,juma,fatuma.juma@work.net,2022-04-10,tanzania,dar es salaam,80000,tzs,31.0,2557551234560,2025-07-15
3,cust005,david,okello,david.okello@email.com,2022-05-12,ug,kla,75000,shs,34.0,07721234560,NaT
6,cust007,john,ochieng,john.ochieng@mail,NaT,ke,niarobi,2500,ksh,50.0,2547221234560,NaT



### Step 8: Outlier Detection and Handling
We’ll visualize numerical columns with boxplots to check for extreme outliers.


In [27]:
result = df[df['Age'] >= 120][['FirstName', 'LastName', 'Age']].copy()
result['Age'] = result['Age'].astype(int)
print(result)

   FirstName LastName           Age
7        bob     kato           150
12     grace   mutesi  250789123456
40      eric  sentamu           250


In [28]:
age_above_100 = df[df['Age'] > 100]


df[df['Age'] >= 100]


,CustomerID,FirstName,LastName,Email,RegistrationDate,Country,City,TotalSpent,Currency,Age,PhoneNumber,LastLoginDate
7,cust009,bob,kato,bob@email.com,2023-05-15,uganda,kmapala,95000,ugx,1.500000e+02,2567521234560,2025-08-18
12,cust016,grace,mutesi,grace.m@mail.com,2024-12-01,rwanda,kigali,40000,-5,2.507891e+11,,NaT
40,cust059,eric,sentamu,eric@email.com,2023-05-15,uganda,kmapala,99000,ugx,2.500000e+02,07561234560,2025-08-18


In [29]:
print("--- Initial DataFrame showing missing 'Age' ---")
print(df)

# Impute missing Age values with the median age

age_median = df['Age'].median()
df['Age'].fillna(age_median, inplace=True)

print("\n--- After Imputing Missing Age with the Median ---")
print(df)
print("\nDataFrame Info (after imputing age):")
df.info()

# This creates a new column to indicate if the Age was missing.
df['Age_missing'] = df['Age'].isna().astype(int)


--- Initial DataFrame showing missing 'Age' ---
   CustomerID FirstName    LastName                   Email RegistrationDate  \
0     cust001     david      okello  david.okello@email.com       2022-01-15   
1     cust002    esther     wanjiku      esther.w@email.com              NaT   
2     cust004    fatuma        juma    fatuma.juma@work.net       2022-04-10   
3     cust005     david      okello  david.okello@email.com       2022-05-12   
6     cust007      john     ochieng       john.ochieng@mail              NaT   
..        ...       ...         ...                     ...              ...   
63    cust092     nancy       mumbi           nancy.m@email       2023-08-11   
64    cust093   stanley        kato     stanley.k@email.com       2023-09-14   
65    cust094      rose      akinyi        rose.a@email.com       2024-10-30   
66    cust096    hellen  chepkwemoi       hellen.c@mail.com       2024-12-01   
67    cust098    vivian     wachira                vivian.w       2025-0

/tmp/ipykernel_28435/1049123966.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(age_median, inplace=True)


In [30]:

# Example: Cap outliers at 1st and 99th percentile
for col in df.select_dtypes(include=[np.number]).columns:
    lower, upper = df[col].quantile([0.01, 0.99])
    df[col] = np.clip(df[col], lower, upper)



### Step 9: Feature Engineering
Let’s create some useful new features, e.g.:
- `order_month` from `order_date`  
- `is_high_value_order` if price > threshold  


In [31]:

if "order_date" in df.columns:
    df['order_month'] = df['order_date'].dt.to_period('M')

if "price" in df.columns:
    df['is_high_value_order'] = df['price'] > df['price'].median()

df.head()


,CustomerID,FirstName,LastName,Email,RegistrationDate,Country,City,TotalSpent,Currency,Age,PhoneNumber,LastLoginDate,Age_missing
0,cust001,david,okello,david.okello@email.com,2022-01-15,uganda,kampala,150000.0,ugx,34.0,2567721234560,2025-08-20,0
1,cust002,esther,wanjiku,esther.w@email.com,NaT,kenya,nairobi,5500.0,kes,28.0,07123456780,2025-08-21,0
2,cust004,fatuma,juma,fatuma.juma@work.net,2022-04-10,tanzania,dar es salaam,80000.0,tzs,31.0,2557551234560,2025-07-15,0
3,cust005,david,okello,david.okello@email.com,2022-05-12,ug,kla,75000.0,shs,34.0,07721234560,NaT,0
6,cust007,john,ochieng,john.ochieng@mail,NaT,ke,niarobi,2500.0,ksh,50.0,2547221234560,NaT,0



### Step 10: Export Cleaned Dataset
Now that cleaning is complete, let’s save the cleaned file.


In [32]:

output_file = "eastafrican-ecommerce-dataset.cleaned.csv"
df.to_csv(output_file, index=False)
print(f"Cleaned dataset saved as {output_file}")


Cleaned dataset saved as eastafrican-ecommerce-dataset.cleaned.csv
